## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [4]:
from CIFAR10_resnet import resnet_v1 # 這是從 CIFAR10_resnet.py 中直接 import 撰寫好的 resnet_v1 函數
model = resnet_v1(input_shape=(32,32,3),depth=20)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 26s 515us/step - loss: 0.5843 - acc: 0.8534 - val_loss: 1.1514 - val_acc: 0.7073
Epoch 2/30
50000/50000 [==============================] - 22s 447us/step - loss: 0.5477 - acc: 0.8685 - val_loss: 0.8556 - val_acc: 0.7725
Epoch 3/30
50000/50000 [==============================] - 22s 442us/step - loss: 0.5132 - acc: 0.8799 - val_loss: 1.1647 - val_acc: 0.7270
Epoch 4/30
50000/50000 [==============================] - 22s 436us/step - loss: 0.4843 - acc: 0.8909 - val_loss: 1.0550 - val_acc: 0.7409
Epoch 5/30
50000/50000 [==============================] - 22s 433us/step - loss: 0.4627 - acc: 0.9015 - val_loss: 1.1480 - val_acc: 0.7280
Epoch 6/30
50000/50000 [==============================] - 22s 432us/step - loss: 0.4391 - acc: 0.9107 - val_loss: 1.0832 - val_acc: 0.7256
Epoch 7/30
50000/50000 [==============================] - 22s 432us/step - loss: 0.4188 - acc: 0.9197 - val_loss: 1.

In [8]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

In [10]:
from keras.optimizers import RMSprop, Adam
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit_generator(data_generator.flow(x_train, y_train,batch_size=batch_size),
                    #batch_size=batch_size,
                    steps_per_epoch=int(len(x_train)/32),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
1562/1562 [==============================] - 41s 26ms/step - loss: 0.8091 - acc: 0.8047 - val_loss: 0.9808 - val_acc: 0.7612
Epoch 2/30
1562/1562 [==============================] - 38s 24ms/step - loss: 0.7154 - acc: 0.8276 - val_loss: 0.9211 - val_acc: 0.7844
Epoch 3/30
1562/1562 [==============================] - 38s 24ms/step - loss: 0.6701 - acc: 0.8404 - val_loss: 0.8191 - val_acc: 0.7982
Epoch 4/30
1562/1562 [==============================] - 38s 25ms/step - loss: 0.6378 - acc: 0.8490 - val_loss: 0.7976 - val_acc: 0.8125
Epoch 5/30
1562/1562 [==============================] - 38s 25ms/step - loss: 0.6155 - acc: 0.8549 - val_loss: 0.9363 - val_acc: 0.7755
Epoch 6/30
1562/1562 [==============================] - 38s 25ms/step - loss: 0.5974 - acc: 0.8587 - val_loss: 0.8118 - val_acc: 0.7928
Epoch 7/30
1562/1562 [==============================] - 38s 25ms/step - loss: 0.5813 - acc: 0.8645 - val_loss: 0.8001 - val_acc: 0.8090
Epoch 8/30
1562/1562 [==========================

In [13]:
from keras.optimizers import RMSprop, Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr = 1e-3),
              metrics=['accuracy'])

history = model.fit_generator(data_generator.flow(x_train, y_train,batch_size=batch_size),
                    #batch_size=batch_size,
                    steps_per_epoch=int(len(x_train)/32),
                    epochs=100,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
1562/1562 [==============================] - 47s 30ms/step - loss: 0.4593 - acc: 0.9044 - val_loss: 0.7864 - val_acc: 0.8191
Epoch 2/100
1562/1562 [==============================] - 44s 28ms/step - loss: 0.4528 - acc: 0.9051 - val_loss: 0.7116 - val_acc: 0.8400
Epoch 3/100
1562/1562 [==============================] - 43s 28ms/step - loss: 0.4526 - acc: 0.9056 - val_loss: 0.8001 - val_acc: 0.8208
Epoch 4/100
1562/1562 [==============================] - 43s 28ms/step - loss: 0.4516 - acc: 0.9046 - val_loss: 0.6706 - val_acc: 0.8487
Epoch 5/100
1562/1562 [==============================] - 43s 27ms/step - loss: 0.4453 - acc: 0.9082 - val_loss: 0.7420 - val_acc: 0.8306
Epoch 6/100
1562/1562 [==============================] - 43s 27ms/step - loss: 0.4445 - acc: 0.9080 - val_loss: 0.7955 - val_acc: 0.8195
Epoch 7/100
1562/1562 [==============================] - 43s 27ms/step - loss: 0.4452 - acc: 0.9071 - val_loss: 0.6221 - val_acc: 0.8580
Epoch 8/100
1562/1562 [==================

1562/1562 [==============================] - 42s 27ms/step - loss: 0.3977 - acc: 0.9248 - val_loss: 0.6239 - val_acc: 0.8653
Epoch 61/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3977 - acc: 0.9247 - val_loss: 0.8006 - val_acc: 0.8299
Epoch 62/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3985 - acc: 0.9241 - val_loss: 0.6079 - val_acc: 0.8736
Epoch 63/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3963 - acc: 0.9241 - val_loss: 0.6485 - val_acc: 0.8689
Epoch 64/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3956 - acc: 0.9252 - val_loss: 0.6124 - val_acc: 0.8743
Epoch 65/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3975 - acc: 0.9239 - val_loss: 0.5818 - val_acc: 0.8739
Epoch 66/100
1562/1562 [==============================] - 42s 27ms/step - loss: 0.3963 - acc: 0.9239 - val_loss: 0.6058 - val_acc: 0.8687
Epoch 67/100
1562/1562 [=======================